In [1]:
import cProfile
import pathlib
import pstats
from io import StringIO

import numpy as np
from gymnasium import spaces
from stable_baselines3.common.utils import set_random_seed

import scenic
from scenic.gym import ScenicGymEnv
from scenic.simulators.metadrive import MetaDriveSimulator


/Users/oliverchang/Documents/RARLET/Scenic/src/scenic/core/errors.py:271: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn("unable to install sys.excepthook to format Scenic backtraces")


# Scenic

## Setup

In [2]:
def make_env() -> callable:
    """Create a function that returns a new environment instance."""

    def thunk() -> ScenicGymEnv:
        scenario = scenic.scenarioFromFile(
            "../scenarios/protagonsit.scenic",
            model="scenic.simulators.metadrive.model",
            mode2D=True,
        )

        env = ScenicGymEnv(
            scenario,
            MetaDriveSimulator(timestep=0.02, sumo_map=pathlib.Path("../maps/Town06.net.xml"), render=False, real_time=False),
            observation_space=spaces.Box(low=-np.inf, high=np.inf, shape=(258,)),
            action_space=spaces.Box(low=-1, high=1, shape=(2,)),
            max_steps=700,
        )
        return env

    return thunk

In [3]:
scenario = scenic.scenarioFromFile(
    "baseline-av.scenic",
    model="scenic.simulators.metadrive.model",
    mode2D=True,
)

env = ScenicGymEnv(
    scenario,
    MetaDriveSimulator(timestep=0.02, sumo_map=pathlib.Path("../maps/Town06.net.xml"), render=False, real_time=False),
    observation_space=spaces.Box(low=-np.inf, high=np.inf, shape=(258,)),
    action_space=spaces.Box(low=-1, high=1, shape=(2,)),
    max_steps=600,
)

/Users/oliverchang/Documents/RARLET/Scenic/src/scenic/syntax/veneer.py:1526: UserWarning: While sampling internally to determine if a random region provides an orientation, the following exception was raised: AssertionError()
  warnings.warn(


## cProfile

In [4]:
ob, _ = env.reset()  # first reset call to initialize the environment

In [5]:
# Profile reset function
profiler = cProfile.Profile()
profiler.enable()

env.reset()  # Function call you to profile

profiler.disable()
s = StringIO()
sortby = "cumulative"
ps = pstats.Stats(profiler, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         1519049 function calls (1501662 primitive calls) in 1.043 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.043    0.522 /Users/oliverchang/opt/anaconda3/envs/tracr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3511(run_code)
        2    0.000    0.000    1.043    0.522 {built-in method builtins.exec}
        1    0.000    0.000    1.043    1.043 /var/folders/w0/3ttb6cs14rq8l4ggd5r1v9440000gn/T/ipykernel_45054/503302022.py:5(<cell line: 5>)
        1    0.000    0.000    1.043    1.043 /Users/oliverchang/Documents/RARLET/Scenic/src/scenic/gym/envs/scenic_gym.py:76(reset)
      2/1    0.000    0.000    1.043    1.043 {method 'throw' of 'generator' objects}
        1    0.000    0.000    1.043    1.043 /Users/oliverchang/Documents/RARLET/Scenic/src/scenic/gym/envs/scenic_gym.py:39(_make_run_loop)
        2    0.000    0.000    1.031    0.516 /Users/oliverchang/Document

# Pure MetaDrive

## Setup

In [1]:
import cProfile
import pathlib
import pstats
from io import StringIO

from metadrive.envs import BaseEnv
from metadrive.manager.sumo_map_manager import SumoMapManager
from metadrive.obs.state_obs import LidarStateObservation


# %%
class DriveEnv(BaseEnv):
    def reward_function(self, agent):
        """Dummy reward function."""
        return 0, {}

    def cost_function(self, agent):
        """Dummy cost function."""
        return 0, {}

    def done_function(self, agent):
        """Dummy done function."""
        return False, {}

    def get_single_observation(self):
        """Dummy observation function."""
        return LidarStateObservation(self.config)

    def setup_engine(self):
        """Setup the engine for MetaDrive."""
        super().setup_engine()
        self.engine.register_manager("map_manager", SumoMapManager("../maps/Town06.net.xml"))


# %%
vehicle_config = {}
vehicle_config["spawn_position_heading"] = [
    (0.0, 0.0),
    0.0,
]
vehicle_config["lane_line_detector"] = dict(
    num_lasers=10,
    distance=20,
)
client = DriveEnv(
    dict(
        vehicle_config=vehicle_config,
        use_render=False,
        use_mesh_terrain=False,
    )
)
client.config["sumo_map"] = "../maps/Town06.net.xml"

[INFO] Environment: DriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None


## cProfile

In [2]:
obs, _ = client.reset() # first reset call to initialize the environment

[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[WARNING] Can not find `start_seed` or `start_scenario_index`. Use 0 as `start_seed` (base_engine.py:582)
[INFO] Start Scenario Index: 0, Num Scenarios : 1
[WARNING] Can not find `start_seed` or `start_scenario_index`. Use 0 as `start_seed` (base_engine.py:582)


In [3]:
# Profile reset
profiler = cProfile.Profile()
profiler.enable()

client.reset()

profiler.disable()
s = StringIO()
sortby = "cumulative"
ps = pstats.Stats(profiler, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[WARNING] Can not find `start_seed` or `start_scenario_index`. Use 0 as `start_seed` (base_engine.py:582)


         18225 function calls (17818 primitive calls) in 0.070 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.070    0.035 /Users/oliverchang/opt/anaconda3/envs/tracr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3511(run_code)
        2    0.000    0.000    0.070    0.035 {built-in method builtins.exec}
        1    0.000    0.000    0.070    0.070 /var/folders/w0/3ttb6cs14rq8l4ggd5r1v9440000gn/T/ipykernel_45347/2457362763.py:5(<cell line: 5>)
        1    0.000    0.000    0.070    0.070 /Users/oliverchang/Documents/tracr/metadrive/metadrive/envs/base_env.py:513(reset)
        1    0.000    0.000    0.064    0.064 /Users/oliverchang/Documents/tracr/metadrive/metadrive/engine/base_engine.py:316(reset)
        3    0.000    0.000    0.050    0.017 /Users/oliverchang/Documents/tracr/metadrive/metadrive/base_class/base_object.py:237(detach_from_world)
        1    0.000    0.